In [33]:
import pandas as pd

def pulisci_dataset_r6_finale(input_file, output_file):
    print("Caricamento del file in corso...")
    
    # Leggiamo il CSV (Pandas userà automaticamente la prima riga come intestazione)
    try:
        df = pd.read_csv(input_file)
    except Exception as e:
        print(f"Errore nell'apertura del file: {e}")
        return

    # ---------------------------------------------------------
    # FASE 0: FILTRO PRELIMINARE (GameMode)
    # ---------------------------------------------------------
    # Filtriamo subito per tenere solo "BOMB".
    # Convertiamo in maiuscolo per evitare errori (es. "Bomb" vs "BOMB")
    if 'gamemode' in df.columns:
        df = df[df['gamemode'].astype(str).str.upper() == 'BOMB']
        print(f"Righe rimanenti dopo filtro BOMB: {len(df)}")
    else:
        print("ATTENZIONE: Colonna 'gamemode' non trovata.")

    # ---------------------------------------------------------
    # FASE 1: RIMOZIONE COLONNE (Armi e altre richieste)
    # ---------------------------------------------------------
    
    # 1. Tagliamo tutto dalla colonna "primaryweapon" in poi
    if 'primaryweapon' in df.columns:
        idx_cut = df.columns.get_loc('primaryweapon')
        df = df.iloc[:, :idx_cut]
    
    # 2. Rimuoviamo dateid, roundduration e ORA ANCHE platform
    cols_to_remove = ['dateid', 'roundduration', 'platform']
    df = df.drop(columns=cols_to_remove, errors='ignore')

    # ---------------------------------------------------------
    # FASE 2: FILTRI GIOCATORI (Livello e Rank)
    # ---------------------------------------------------------
    
    # Conversione clearancelevel in numero
    df['clearancelevel'] = pd.to_numeric(df['clearancelevel'], errors='coerce')

    # Funzione per filtrare i rank indesiderati
    def is_rank_accepted(rank_str):
        if pd.isna(rank_str): return False
        rank_str = str(rank_str).strip().lower()
        
        # Parole vietate (escludiamo Unranked, Copper e Bronze)
        # Nota: tutto il resto (es. Silver, Gold, Platinum...) sarà accettato
        bad_ranks = ['unranked', 'copper', 'bronze']
        
        for bad in bad_ranks:
            if bad in rank_str:
                return False
        return True

    # Creiamo i flag di validità per ogni singolo giocatore
    df['is_level_ok'] = df['clearancelevel'] >= 50
    df['is_rank_ok'] = df['skillrank'].apply(is_rank_accepted)
    
    # Un giocatore è valido SOLO se livello >= 50 AND rank è ok (>= Silver)
    df['player_is_valid'] = df['is_level_ok'] & df['is_rank_ok']

    # ---------------------------------------------------------
    # FASE 3: FILTRO PER ROUND (Logica 5vs5 completa)
    # ---------------------------------------------------------
    
    print("Analisi dei round per trovare le partite complete (10 giocatori validi)...")

    # Raggruppiamo per matchid e roundnumber per analizzare il round intero
    round_stats = df.groupby(['matchid', 'roundnumber']).agg(
        all_players_valid=('player_is_valid', 'all'),   # True solo se TUTTI i giocatori del round sono validi
        total_players=('player_is_valid', 'count')      # Conta quanti giocatori ci sono nel round
    ).reset_index()

    # Manteniamo solo i round che hanno esattamente 10 giocatori E tutti validi
    valid_rounds = round_stats[
        (round_stats['all_players_valid'] == True) & 
        (round_stats['total_players'] == 10)
    ]

    # Facciamo il merge per tenere solo le righe dei round validi
    df_clean = pd.merge(df, valid_rounds[['matchid', 'roundnumber']], on=['matchid', 'roundnumber'], how='inner')

    # ---------------------------------------------------------
    # FASE 4: PULIZIA FINALE E SALVATAGGIO
    # ---------------------------------------------------------
    
    # Rimuoviamo le colonne temporanee di calcolo
    df_clean = df_clean.drop(columns=['is_level_ok', 'is_rank_ok', 'player_is_valid'])

    print(f"--- Report Finale ---")
    print(f"Righe totali salvate: {len(df_clean)}")
    print(f"Numero di round completi trovati: {len(valid_rounds)}")
    
    df_clean.to_csv(output_file, index=False)
    print(f"File pulito salvato come: {output_file}")

# ESEMPIO DI ESECUZIONE
if __name__ == "__main__":
    input_csv = 'csv/datadump_s5-002.csv'   # Sostituisci con il nome del tuo file
    output_csv = 'csvpuliti/datadump_s5-002_pulito.csv'
    
    pulisci_dataset_r6_finale(input_csv, output_csv)

Caricamento del file in corso...
Righe rimanenti dopo filtro BOMB: 993431
Analisi dei round per trovare le partite complete (10 giocatori validi)...
--- Report Finale ---
Righe totali salvate: 94500
Numero di round completi trovati: 9450
File pulito salvato come: csvpuliti/datadump_s5-002_pulito.csv
